# 载入数据

打开cuda

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tensorboard import program
import torch
writer = SummaryWriter(log_dir='./log/train/') #日志存储的路径
device = torch.device("cuda")

划分训练集，测试集

In [ ]:
import numpy as np
import pandas as pd
import glob
labelDict = {'1': 0,'2':1,'3':2,'G11': 3,'G12':4,'G13':5}
folders = glob.glob('indata/npy224/*')
# 数据
datas = []
for i in range(6):
    datas.append([])
    pass

# 载入数据
for folder in folders:      
    folder = folder.replace('\\', '/')
    label = labelDict[folder.split('/')[-1].split('_')[0]]
    # 数据提取
    npys = glob.glob(folder + '/*.npy')
    for npy in npys:
        npy = npy.replace('\\', '/')
        # data_npy = np.load(file=npy)
        datas[label].append(npy)    
        pass
    pass
datas
datanp = np.array(datas)
datanp.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# datanp=shuffle(datanp) 
train_data = []
train_label = []
test_data = []
test_label = []

for label in range(6):
    traindata,testdata = train_test_split(datanp[label],test_size=0.2,random_state=0,shuffle=True)
    train_data.extend(traindata)
    test_data.extend(testdata)
    train_shape = np.array(traindata).shape
    test_shape = np.array(testdata).shape
    train_label.extend(np.full(train_shape,label))
    test_label.extend(np.full(test_shape,label))
     
# train_data = np.array(train_data)
# train_label = np.array(pd.get_dummies(train_label))
# test_data = np.array(test_data)
# test_label = np.array(pd.get_dummies(test_label))

train_pd = pd.DataFrame(columns=['datapath','label'])
test_pd = pd.DataFrame(columns=['datapath','label'])
for i in range(0,len(train_data)):
    train_pd = train_pd.append(pd.Series({'datapath':train_data[i],'label':train_label[i]}),ignore_index=True)
    pass
for i in range(0,len(test_data)):
    test_pd = test_pd.append(pd.Series({'datapath':test_data[i],'label':test_label[i]}),ignore_index=True)
    pass
train_pd = shuffle(train_pd)
test_pd = shuffle(test_pd)
# train_pd = pd.concat([pd.DataFrame(train_data),pd.DataFrame(train_label)],axis=1)
# test_pd = pd.concat([pd.DataFrame(test_data),pd.DataFrame(test_label)],axis=1)
train_data = np.array(train_pd['datapath'])
train_label = np.array(pd.get_dummies(train_pd['label']))
test_data = np.array(test_pd['datapath'])
test_label = np.array(pd.get_dummies(test_pd['label']))
# test_label = np.array(test_label)
# 训练模型
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)


In [ ]:


np.savetxt('train2.txt', train_data,fmt='%s', delimiter=',', encoding='utf-8')

使用之前的划分结果

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch
from tqdm import tqdm
import torchvision.utils
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import dataset
import torch.nn
class LoadData(dataset.Dataset):

    def __init__(self,data_paths,is_train = True):
        super(LoadData, self).__init__()
        self.data_paths = data_paths
        self.is_train = is_train
        self.label_Dict = {'1': 0,'2':1,'3':2,'G11': 3,'G12':4,'G13':5}
        # 数据预处理
        
        pass

    def __getitem__(self,index):
        data_path = self.data_paths[index]
        data_name_arr = data_path.split('\\')
        print(data_name_arr)
        key = data_name_arr[-2].split('_')[0]
        label = self.label_Dict[key]
        data = np.load(data_path)
        data[np.isinf(data)] = 0
        data[np.isnan(data)] = 0
        
        # print(data_path)
        # print(data.shape)
        # 修改维度,1DCNN注释此行，2D3D不注释
        tensor = data.transpose(2,0,1)
        # 3DCNN增加下面行代码，2DCNN不加
        # tensor = np.expand_dims(data, axis=0)
        return tensor,label

    def __len__(self):
        return len(self.data_paths)

In [ ]:
import glob
from torch.utils.data import dataloader
from torch.utils.data import dataset
f1=open('train2.txt', encoding='utf-8')
f2=open('test2.txt', encoding='utf-8')
path1=[]
path2=[]
for line in f1:
    path1.append(line.strip())
for line in f2:
    path2.append(line.strip())
# 读取train test进入list
train_dataset = LoadData(data_paths=path1,is_train=True)
test_dataset = LoadData(data_paths=path2,is_train=True)

In [ ]:
train_dataset

# 建模

模型

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet2D(nn.Module):
    def __init__(self, width=30, height=30, channel=224):
        super(ConvNet2D, self).__init__()
        self.width = width
        self.height = height
        self.channel = channel

        self.conv1 = nn.Conv2d(in_channels=224, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batch_norm = nn.BatchNorm2d(num_features=64)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(in_features=64*height*width//4, out_features=224)
        self.layer_norm = nn.LayerNorm(normalized_shape=[224])
        self.fc2 = nn.Linear(in_features=224, out_features=6)

    def forward(self, x):
        aver = torch.mean(x, dim=1)
        aver = torch.mean(aver, dim=0)
        aver = torch.mean(aver, dim=1)
        x=x.reshape(1,30,30,224)
        print(x.shape)

        x = self.conv1(x)
        x = F.tanh(self.conv2(x))
        x = self.pool(x)
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = x.view(-1, 64*self.width*self.height//4)
        x = F.glorot_uniform(self.fc1(x))
        x = self.layer_norm(x + aver)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x



In [ ]:
from torchinfo import summary
print(summary(model))

检查设备

In [ ]:
import torch.optim as optim

# Define the device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the Model2D class
model = ConvNet2D().to(device)

开始训练

In [ ]:
import torch
from tqdm import tqdm
import torchvision.utils
from torch.optim.lr_scheduler import ReduceLROnPlateau
# model = torch.load('default+pre_norm+drop=0.5.pt')
# model = model.to(device)
import glob
from torch.utils.data import dataloader
from torch.utils.data import dataset
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,'max',0.7,15,eps=1e-6)
batch_size = 16
train_loader = dataloader.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
    )
test_loader = dataloader.DataLoader(
dataset=test_dataset,
batch_size=batch_size,
shuffle=True
    )
epoch = 200


running_loss = 0.0
for t in range(epoch):
    right = 0
    train_total = 0
    model.train()
    
    loop = tqdm(enumerate(train_loader), total =len(train_loader))
    for step, (x, y) in loop:
        # Forward pass: Compute predicted y by passing x to the model
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        train_total += y.size(0)
        # Compute and print loss
        loss = criterion(y_pred, y) # 计算损失函数

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度
        loss.backward() # loss反向传播
        optimizer.step() # 反向传播后参数更新
      
        running_loss += loss.item()
        # 可视化graph
        if t == 0:
            writer.add_graph(model,x)
        # 可视化图像
        # img_grid = torchvision.utils.make_grid(x)
        # writer.add_image('hyperspectral_wheat_seed',img_grid)

        _, predicted = torch.max(y_pred, 1)
        # 累加识别正确的样本数
        right += (predicted == y).sum()
        acc = float(right)/float(batch_size*step+len(x))
        #更新信息
        loop.set_description(f'Epoch [{t}/{epoch}]')
        loop.set_postfix(train_loss=loss.item(), train_acc=acc,lr=optimizer.state_dict()['param_groups'][0]['lr'])
   
        # loop.set_postfix(loss=running_loss/(step+1), acc=acc)
        if step+1 == len(train_loader):
            # ...log the running loss
            writer.add_scalar('training loss',
                            running_loss,
                            epoch * len(train_loader) + step)
            writer.add_scalar('training acc',
                            acc,
                            epoch * len(train_loader) + step)
        # ...log a Matplotlib Figure showing the model's predictions on a
        # random mini-batch
        # writer.add_figure('predictions vs. actuals',
        #                 plot_classes_preds(net, inputs, labels),
        #                 global_step=epoch * len(train_loader) + i)
        running_loss = 0.0
       # 测试模式
    test_correct = 0
    test_total = 0
    test_sum_loss = 0
    
    model.eval()
    with torch.no_grad():

        loop2 = tqdm(test_loader, desc='Test')
        for x, y in loop2:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            y_ = torch.argmax(y_pred, dim=1)
            test_correct += (y_ == y).sum().item()
            test_total += y.size(0)
            test_sum_loss += loss.item()
            test_running_loss = test_sum_loss / test_total
            test_running_acc = test_correct / test_total
            
            # 更新测试信息
            loop2.set_postfix(loss = test_running_loss, acc = test_running_acc)
           
    test_epoch_loss = test_sum_loss / test_total
    test_epoch_acc = test_correct / test_total
    scheduler.step(test_running_acc)
print('Finished Training')
writer.close()